## Training

In [12]:
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, TrainingArguments, Trainer, DataCollatorForSeq2Seq

# Clean CUDA cache
torch.cuda.empty_cache()

# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base')

# Load the dataset
from datasets import load_from_disk
dataset_samsum = load_from_disk('./samsum_dataset')

# Function to convert examples to features
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['summary'], max_length=128, truncation=True)
    return {
        'input_ids': input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }

# Map the conversion function to the dataset
dataset_samsum_pt = dataset_samsum.map(convert_examples_to_features, batched=True)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,  # Reduced number of epochs
    per_device_train_batch_size=1,  # Batch size of 1
    per_device_eval_batch_size=1,  # Batch size of 1
    warmup_steps=100,  # Reduced warmup steps
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,  # Reduced logging steps
    evaluation_strategy="steps",
    eval_steps=100,  # Reduced evaluation steps
    save_steps=200,  # Reduced save steps
    gradient_accumulation_steps=1,  # No gradient accumulation
    fp16=True  # Use mixed precision training
)

# Data collator
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_samsum_pt["train"],
    eval_dataset=dataset_samsum_pt["validation"],
    data_collator=data_collator,
)

# Train the model
trainer.train()

# Save the model
trainer.save_model('./t5_base_trained')


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/home/jellyboi/anaconda3/envs/textS/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 818/818 [00:00<00:00, 3179.94 examples/s]
/home/jellyboi/anaconda3/envs/textS/lib/python3.8/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  0%|          | 51/14732 [00:08<42:46,  5.72it/s] 

{'loss': 3.0774, 'grad_norm': 18.636903762817383, 'learning_rate': 2.25e-05, 'epoch': 0.0}


  1%|          | 100/14732 [00:17<41:58,  5.81it/s]

{'loss': 2.0201, 'grad_norm': 15.437552452087402, 'learning_rate': 4.75e-05, 'epoch': 0.01}


                                                   
  1%|          | 101/14732 [00:28<14:14:16,  3.50s/it]

...

{'loss': 1.4811, 'grad_norm': 5.403878211975098, 'learning_rate': 1.8452706396938218e-07, 'epoch': 1.0}


                                                       
100%|█████████▉| 14701/14732 [1:10:26<01:38,  3.18s/it]

{'eval_loss': 1.330243468284607, 'eval_runtime': 10.0245, 'eval_samples_per_second': 81.6, 'eval_steps_per_second': 81.6, 'epoch': 1.0}


100%|██████████| 14732/14732 [1:10:31<00:00,  3.48it/s]


{'train_runtime': 4231.9966, 'train_samples_per_second': 3.481, 'train_steps_per_second': 3.481, 'train_loss': 1.5405529352623646, 'epoch': 1.0}


## Evaluation

In [17]:
from datasets import load_metric
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Clean CUDA cache
torch.cuda.empty_cache()

# Load the trained model
model_path = './t5_base_trained'
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Load BLEU metric with trust_remote_code=True
bleu = load_metric('bleu', trust_remote_code=True)

def compute_bleu(predictions, references):
    bleu.add_batch(predictions=predictions, references=references)
    result = bleu.compute()
    return result['bleu']

# Function to generate summaries
def generate_summary(dialogue):
    inputs = tokenizer(dialogue, return_tensors="pt", max_length=1024, truncation=True)
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate BLEU score on the test dataset
test_dialogues = [dataset_samsum['test'][i]['dialogue'] for i in range(len(dataset_samsum['test']))]
test_summaries = [dataset_samsum['test'][i]['summary'] for i in range(len(dataset_samsum['test']))]

predicted_summaries = [generate_summary(dialogue) for dialogue in test_dialogues]

bleu_score = compute_bleu(predictions=[pred.split() for pred in predicted_summaries],
                         references=[[ref.split()] for ref in test_summaries])
print(f"BLEU score: {bleu_score}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


BLEU score: 0.13333111566594982


In [19]:
from datasets import load_metric
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Clean CUDA cache
torch.cuda.empty_cache()

# Load the trained model
model_path = './t5_base_trained'
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained(model_path)

# Load ROUGE metric
rouge = load_metric('rouge')

def compute_rouge(predictions, references):
    rouge.add_batch(predictions=predictions, references=references)
    result = rouge.compute()
    return {
        'rouge1': result['rouge1'].mid,
        'rouge2': result['rouge2'].mid,
        'rougeL': result['rougeL'].mid
    }

# Function to generate summaries
def generate_summary(dialogue):
    inputs = tokenizer(dialogue, return_tensors="pt", max_length=1024, truncation=True)
    with torch.no_grad():
        outputs = model.generate(inputs["input_ids"], max_length=128, num_beams=4, early_stopping=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Evaluate ROUGE score on the test dataset
test_dialogues = [dataset_samsum['test'][i]['dialogue'] for i in range(len(dataset_samsum['test']))]
test_summaries = [dataset_samsum['test'][i]['summary'] for i in range(len(dataset_samsum['test']))]

predicted_summaries = [generate_summary(dialogue) for dialogue in test_dialogues]

rouge_scores = compute_rouge(predictions=predicted_summaries, references=test_summaries)
print(f"ROUGE-1: {rouge_scores['rouge1']}")
print(f"ROUGE-2: {rouge_scores['rouge2']}")
print(f"ROUGE-L: {rouge_scores['rougeL']}")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ROUGE-1: Score(precision=0.5960998689074908, recall=0.4502035098873436, fmeasure=0.48343058752187307)
ROUGE-2: Score(precision=0.3073526717509122, recall=0.22752341219904754, fmeasure=0.24603915147122501)
ROUGE-L: Score(precision=0.5021220111476851, recall=0.3793270528013324, fmeasure=0.40781455767281716)


In [26]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load T5 tokenizer and model
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('./t5_base_trained')  # Adjust path to your trained model

# Example dialogue and reference summary
sample_text = """We've updated our terms of use and privacy policy 

Hello Codewarrior,We’ve updated our Terms of Use and Privacy Policy, which took effect on June 14, 
2024. These updates are intended to make it easier for you to understand how we use your information, 
our content standards, and changes to our contact details. 
You can review the full updated Terms of Use and Privacy Policy linked above. We encourage you to read 
them carefully and contact us at info@codewars.com if you have any questions or concerns. 
Some of the key changes include: Explaining how we share your personal information with our affiliates, 
vendors, service providers, partners, and third parties for various purposes. 
Outlining your privacy rights and choices depending on your location and applicable laws.Updating our 
content standards and prohibited uses for the website and the interactive services we offer, such as 
Discussions, Discord, Kata discourse, etc.  
By continuing to use our website or services, you agree to the updated Terms of Use and Privacy Policy. 
The Codewars Team """
reference = """Codewars updated its Terms of Use and Privacy Policy on June 14, 
2024, to clarify information usage, content standards, and contact details. 
Changes include how personal information is shared, privacy rights based on location, 
and updated content standards for website use. Users are encouraged to review the full 
updates and contact Codewars at info@codewars.com for questions"""

# Tokenize the input dialogue
inputs = tokenizer.encode("summarize: " + sample_text, return_tensors="pt", max_length=512, truncation=True)

# Generate summary
summary_ids = model.generate(inputs, max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)
generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Print outputs
print("Dialogue:")
print(sample_text)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(generated_summary)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Dialogue:
We've updated our terms of use and privacy policy 

Hello Codewarrior,We’ve updated our Terms of Use and Privacy Policy, which took effect on June 14, 
2024. These updates are intended to make it easier for you to understand how we use your information, 
our content standards, and changes to our contact details. 
You can review the full updated Terms of Use and Privacy Policy linked above. We encourage you to read 
them carefully and contact us at info@codewars.com if you have any questions or concerns. 
Some of the key changes include: Explaining how we share your personal information with our affiliates, 
vendors, service providers, partners, and third parties for various purposes. 
Outlining your privacy rights and choices depending on your location and applicable laws.Updating our 
content standards and prohibited uses for the website and the interactive services we offer, such as 
Discussions, Discord, Kata discourse, etc.  
By continuing to use our website or services, 